#### Projeto C318 - Detecção de Fraudes em Cartões Bancários


#### Conjunto de Dados

#### Utilização do Isolation Forest Para isolar as Possíveis Anomalias

In [4]:
!pip install pandas scikit-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

# Carregar o conjunto de dados
df = pd.read_csv("creditcard.csv")

# Selecionar as colunas de features e a coluna de classe
X = df.drop(columns=['Class'])
y = df['Class']

# Aplicar PCA para reduzir para 20 componentes principais
pca = PCA(n_components=20)
X_pca = pca.fit_transform(X)

# Etapa 1: Detecção de anomalias com Isolation Forest
iso_forest = IsolationForest(contamination=0.02, random_state=42)
anomaly_labels = iso_forest.fit_predict(X_pca)

# Adicionar os rótulos de anomalia ao dataframe original
df['anomaly'] = anomaly_labels

# Selecionar apenas os outliers detectados (anomaly == -1)
outliers = df[df['anomaly'] == -1]
X_outliers = outliers.drop(columns=['Class', 'anomaly'])
y_outliers = outliers['Class']  # Classes reais dos outliers

# Calcular Precisão e Revocação
TP = ((df['anomaly'] == -1) & (df['Class'] == 1)).sum()   # Verdadeiros Positivos
FP = ((df['anomaly'] == -1) & (df['Class'] == 0)).sum()   # Falsos Positivos
FN = ((df['anomaly'] == 1) & (df['Class'] == 1)).sum()  # Falsos Negativos

precisionISO = TP / (TP + FP) if (TP + FP) > 0 else 0
recallISO = TP / (TP + FN) if (TP + FN) > 0 else 0

print(f"Precisão somente com o Isolation Forest: {precisionISO:.2f}")
print(f"Revocação somente com o Isolation Forest: {recallISO:.2f}")

# Etapa 2: Treinamento do Classificador Supervisionado
# Dividir os dados de outliers em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_outliers, y_outliers, test_size=0.3, random_state=42)

# Treinar um classificador supervisionado (Random Forest)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Calcular métricas de avaliação
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Resultados
print(f"Precisão após classificador supervisionado: {precision:.2f}")
print(f"Revocação após classificador supervisionado: {recall:.2f}")


Precisão somente com o Isolation Forest: 0.07
Revocação somente com o Isolation Forest: 0.85
Precisão após classificador supervisionado: 0.95
Revocação após classificador supervisionado: 0.91
